In [1]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [2]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
headers = {"X-Naver-Client-Id": "376ZADkaoIoxnJh_ixoG",
           "X-Naver-Client-Secret":"FEja8f5GwD"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [3]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"",
    "ages":["60"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='60'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='60'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='60'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["디지털/가전"], axis=1, inplace=True)
df_all_a

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,4.24599,0.46401,3.14284,1.54432,16.15987,7.54202,0.74528,0.44161,1.20751,0.39369,...,4.98009,0.74528,0.32024,1.73467,0.56926,0.12764,0.10659,PC+MO,전체,60
2021-05-11,4.13537,0.39548,2.77423,1.42160,14.24335,6.83614,0.65391,0.40758,1.20079,0.37936,...,4.56489,0.65391,0.36279,1.53984,0.43490,0.14601,0.06270,PC+MO,전체,60
2021-05-12,4.28765,0.42594,2.37337,1.45743,14.39026,6.99738,0.66735,0.37398,1.13719,0.37130,...,4.60341,0.66735,0.46311,1.50043,0.41832,0.14287,0.07390,PC+MO,전체,60
2021-05-13,4.00324,0.44609,2.35411,1.44355,14.53358,9.38285,0.57598,0.42459,1.14883,0.41698,...,4.58773,0.57598,0.29650,1.49147,0.39459,0.15317,0.06942,PC+MO,전체,60
2021-05-14,3.92530,0.40220,2.28110,1.34904,13.55853,11.38671,0.57195,0.37577,1.14346,0.30546,...,4.30601,0.57195,0.25395,1.36830,0.45550,0.11734,0.06180,PC+MO,전체,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,7.47706,0.64715,3.23414,2.12118,18.22190,19.85910,0.62014,0.32523,1.23313,0.56778,...,5.78309,0.62014,0.47131,1.87808,0.40351,0.24750,0.13229,MO,전체,60
2023-06-30,6.93078,0.62896,3.06381,2.32735,17.23793,17.43307,0.66094,0.31641,1.10634,0.47737,...,5.17617,0.66094,0.43162,1.75791,0.41398,0.21112,0.12623,MO,전체,60
2023-07-01,7.76316,0.56392,3.12389,2.51752,18.68660,21.08286,0.77063,0.34177,1.05011,0.36437,...,5.68883,0.77063,0.50824,1.79650,0.50383,0.30318,0.17750,MO,전체,60


In [4]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"f",
    "ages":["60"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='60'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='60'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='60'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["디지털/가전"], axis=1, inplace=True)
df_all_b

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,2.98882,0.33434,1.63487,1.57869,19.13586,6.52292,0.39421,0.23026,0.44118,0.21552,...,2.35514,0.39421,0.22842,0.44947,0.65026,0.06355,0.07276,PC+MO,여성,60
2021-05-11,2.94553,0.31500,1.50500,1.42855,17.20809,5.88647,0.46697,0.28736,0.45868,0.23486,...,2.03000,0.46697,0.24684,0.42368,0.44763,0.01657,0.04144,PC+MO,여성,60
2021-05-12,3.17948,0.32236,1.37605,1.56487,16.83138,6.14989,0.38500,0.23394,0.47618,0.20355,...,2.26671,0.38500,0.34355,0.40894,0.42000,0.01473,0.05710,PC+MO,여성,60
2021-05-13,3.00632,0.28644,1.29961,1.67355,17.07822,8.44055,0.38960,0.29105,0.55171,0.21276,...,2.17092,0.38960,0.23579,0.42000,0.39605,0.02671,0.04789,PC+MO,여성,60
2021-05-14,3.01001,0.22289,1.31250,1.38342,16.25940,10.36280,0.35644,0.23302,0.46237,0.17868,...,2.00606,0.35644,0.15197,0.28460,0.45131,0.01934,0.05342,PC+MO,여성,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,4.77593,0.47861,1.61174,2.23865,21.08465,18.67623,0.21680,0.15033,0.33441,0.16669,...,2.22944,0.21680,0.33134,0.42748,0.46020,0.08488,0.07976,MO,여성,60
2023-06-30,4.51310,0.38248,1.50641,2.63545,19.69176,16.34043,0.32316,0.11965,0.35896,0.17487,...,1.86844,0.32316,0.33441,0.44793,0.37839,0.05931,0.08386,MO,여성,60
2023-07-01,5.27090,0.28021,1.56470,2.71624,21.07954,18.59646,0.31907,0.11249,0.32112,0.19021,...,2.14354,0.31907,0.34771,0.54406,0.43770,0.07772,0.09715,MO,여성,60


In [5]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"m",
    "ages":["60"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='60'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='60'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='60'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["디지털/가전"], axis=1, inplace=True)
df_all_c

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,4.36477,0.47113,3.66963,1.21387,10.71348,6.83033,0.86525,0.51523,1.55199,0.45152,...,5.99378,0.86525,0.32692,2.36965,0.39552,0.15120,0.11130,PC+MO,남성,60
2021-05-11,4.22477,0.37872,3.19220,1.13617,9.18318,6.21079,0.66714,0.41862,1.52819,0.41442,...,5.59195,0.66714,0.37942,2.08473,0.33952,0.21561,0.06650,PC+MO,남성,60
2021-05-12,4.28497,0.42072,2.66367,1.08856,9.69912,6.26260,0.75044,0.40672,1.41549,0.42562,...,5.47225,0.75044,0.46272,2.03433,0.33462,0.21211,0.07210,PC+MO,남성,60
2021-05-13,3.97205,0.47953,2.69167,0.98426,9.73552,8.25002,0.60413,0.44242,1.37628,0.49003,...,5.52055,0.60413,0.28421,2.01192,0.31572,0.21911,0.07210,PC+MO,남성,60
2021-05-14,3.84744,0.45922,2.56776,1.05706,8.83386,9.92103,0.62303,0.41022,1.43579,0.34162,...,5.20553,0.62303,0.28141,1.92232,0.36892,0.16871,0.05600,PC+MO,남성,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,8.33528,0.66246,4.00859,1.55295,11.63074,16.64054,0.85670,0.41568,1.79129,0.81354,...,7.79855,0.85670,0.49825,2.80188,0.26461,0.34343,0.15201,MO,남성,60
2023-06-30,7.64748,0.71876,3.83124,1.53794,11.24320,14.66252,0.82667,0.42600,1.55389,0.65120,...,7.08917,0.82667,0.42788,2.58043,0.35750,0.30496,0.13793,MO,남성,60
2023-07-01,8.36906,0.70281,3.87441,1.78754,12.44991,18.79310,1.01810,0.47855,1.49290,0.44571,...,7.70753,1.01810,0.54611,2.55510,0.45603,0.44477,0.21300,MO,남성,60


In [6]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6989 entries, 2021-05-10 to 2023-07-03
Data columns (total 28 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   휴대폰액세서리    6989 non-null   float64
 1   생활가전       6989 non-null   float64
 2   영상가전       6989 non-null   float64
 3   이미용가전      6989 non-null   float64
 4   주방가전       6989 non-null   float64
 5   계절가전       6989 non-null   float64
 6   카메라/캠코더용품  6989 non-null   float64
 7   PC         6989 non-null   float64
 8   PC부품       6989 non-null   float64
 9   PC액세서리     6989 non-null   float64
 10  저장장치       6989 non-null   float64
 11  멀티미디어장비    6989 non-null   float64
 12  모니터        6989 non-null   float64
 13  모니터주변기기    6989 non-null   float64
 14  노트북        6989 non-null   float64
 15  노트북액세서리    6989 non-null   float64
 16  소프트웨어      6989 non-null   float64
 17  주변기기       6989 non-null   float64
 18  음향가전       6989 non-null   float64
 19  태블릿PC      6989 non-null   float64
 20

In [7]:
## 엑셀에 저장
df_all.to_excel("MCLS_04_all_fm_60.xlsx", sheet_name="click ratio")
df_all.head()

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,4.24599,0.46401,3.14284,1.54432,16.15987,7.54202,0.74528,0.44161,1.20751,0.39369,...,4.98009,0.74528,0.32024,1.73467,0.56926,0.12764,0.10659,PC+MO,전체,60
2021-05-11,4.13537,0.39548,2.77423,1.42160,14.24335,6.83614,0.65391,0.40758,1.20079,0.37936,...,4.56489,0.65391,0.36279,1.53984,0.43490,0.14601,0.06270,PC+MO,전체,60
2021-05-12,4.28765,0.42594,2.37337,1.45743,14.39026,6.99738,0.66735,0.37398,1.13719,0.37130,...,4.60341,0.66735,0.46311,1.50043,0.41832,0.14287,0.07390,PC+MO,전체,60
2021-05-13,4.00324,0.44609,2.35411,1.44355,14.53358,9.38285,0.57598,0.42459,1.14883,0.41698,...,4.58773,0.57598,0.29650,1.49147,0.39459,0.15317,0.06942,PC+MO,전체,60
2021-05-14,3.92530,0.40220,2.28110,1.34904,13.55853,11.38671,0.57195,0.37577,1.14346,0.30546,...,4.30601,0.57195,0.25395,1.36830,0.45550,0.11734,0.06180,PC+MO,전체,60
